# Element de base pour HFE 

In [188]:
p = 2
m = 1
q = p^m
K.<alpha> = GF(q)
N = 20
L_N.<a> = GF(q^N)
P = L_N._modulus

In [189]:
x = ['x'+str(k) for k in range(N)]
M = PolynomialRing(L_N,N,x)
I = M.ideal([M(x[i])^q - M(x[i]) for i in range(N)])
Q = M.quotient(I)

R.<X> = PolynomialRing(M)
S = R.quotient(R(P))

variable = sum([M(x[i])*X^i for i in range(N)])

def HFE_representation_polynome(f):
    """
    Entrée: f, un polynôme de K[X] qui vérifie les conditions d'HFE
    Sortie: (p0,p1,..,pn), polynôme de Fq[x0,..,xn] tel que 
            f(x0,..,xn) = (p0(x0,..,xn),.., pn(x0,..xn))
    """
    f_prime = R(f)
    p_list = [Q(i).lift() for i in list(S(f_prime(variable)))]
    return p_list

In [190]:
def test_HFE_rep():
    T.<Y> = PolynomialRing(L_N)
    f = R(T.random_element(5))
    return representation_polynome_HFE(f)

#test_HFE_rep()

# Mise en place du cryptosystème HFE

In [191]:
def HFE_function(d, number_coeff):
    """
    Entrée: d, degrée de f qui est somme de deux puissances de q
            number_coeff, nombre de coefficient non nul de f au plus
    Sortie: f, polynôme valide pour HFE
    """
    
    coeff = [L_N.random_element() for i in range(number_coeff)]
    f = 0
    lim = floor(log(d,q)-1)
    for i in range(len(coeff)-2):
        theta = randint(0,lim)
        phi = randint(0,lim)
        f += coeff[i] * X^(q^theta + q^phi)
        
    mu_0 =  L_N.random_element()
    f += mu_0 + coeff[-1] * X^d 
    return f

In [193]:
def HFE_generates_keys(d, number_coeff):
    f = HFE_function(d, number_coeff)
    
    T.<Y> = PolynomialRing(L_N)

    s = R(T.random_element(1))
    t = R(T.random_element(1))
    
    p_list = HFE_representation_polynome(s(f(t)))
    private_key = (L_N,f,s,t)
    public_key = (K, p_list)
    return public_key, private_key 

In [194]:
# example 1 page 5 Patarin avec le clair
def HFE_redundancy(m):
    """ TO DO """
# example 2 Patarin avec le chiffré
def HFE_redundancy_2(y):
    """ TO DO """
    

In [195]:
def HFE_encryption(m, public_key):
    """
    Entrée: m, on supposera qu'il s'agit d'un élément de (F_q)^n
    Sortie: c, c = HFE(m)
    """
    #redundancy = HFE_redundancy(m)
    K, p_list = public_key
    m = tuple(m)
    return [p(m) for p in p_list]

In [196]:
def HFE_decryption(y, private_key):
    L_N, f, s, t = private_key
    a = L_N([0,1])
    y = sum([y[i]*a**i for i in range(N)])

    list_s = list(s)
    s_inv = list_s[1]**(-1) * (X - list_s[0])
    list_t = list(t)
    t_inv = list_t[1]**(-1) * (X - list_t[0])
        
    ft = s_inv(y)
    
    v = tuple((L_N^N).zero())
    
    roots = (f-ft).roots(multiplicities = False)
    m = [L_N.vector_space(map = false)((t_inv(r))(v)) for r in roots]
   
    return m

In [202]:
public_key, private_key = HFE_generates_keys(20, 3)

In [203]:
v = L_N.vector_space(map=False)(L_N.random_element())
p = HFE_encryption(v,public_key)
m = HFE_decryption(p, private_key)
m

[(0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1),
 (1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1),
 (1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1)]

In [206]:
v 

(0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1)